<a href="https://colab.research.google.com/github/sukant16/EIP-Submissions/blob/master/E3P2Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://www.gutenberg.org/files/11/11-0.txt"

--2019-07-26 15:02:31--  https://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173595 (170K) [text/plain]
Saving to: ‘11-0.txt.1’

11-0.txt.1          100%[===================>] 169.53K  --.-KB/s    in 0.1s    

2019-07-26 15:02:31 (1.14 MB/s) - ‘11-0.txt.1’ saved [173595/173595]



In [2]:
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Masking, LSTM, InputLayer
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, Sequence
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:
import os
print(os.getcwd())
!ls

/content
11-0.txt  11-0.txt.1  assgn2_saved_models  sample_data


In [0]:
import re
filename = "11-0.txt"
with open(filename, 'r') as f:
  text = f.read().lower().replace("\n", " ")

# removing punctuations   
text = text.split(".")
text = [re.sub(r'[^\w\s]+', "", line) for line in text]
raw_text = [re.sub(r' +', " ", line) for line in text]

In [5]:
raw_text[:4]

['project gutenbergs alices adventures in wonderland by lewis carroll this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever',
 ' you may copy it give it away or reuse it under the terms of the project gutenberg license included with this ebook or online at www',
 'gutenberg',
 'org title alices adventures in wonderland author lewis carroll posting date june 25 2008 ebook 11 release date march 1994 last updated october 6 2016 language english character set encoding utf8 start of this project gutenberg ebook alices adventures in wonderland alices adventures in wonderland lewis carroll the millennium fulcrum edition 3']

In [6]:
chars = sorted(list({char for line in raw_text for char in line}))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = sum(map(len, raw_text))
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  152579
Total Vocab:  38


In [7]:
# removing those sentences where there are only 1 or 2 characters
raw_text = [line for line in raw_text if len(line)>2]
min(map(len, raw_text))

3

In [0]:
'''
First, I tried that each batch should have some fixed number of sentences and 
then converting them into sequences. However, despite all the hacks I could try, 
it always went OOM due to huge matrix size for some batches. 
So, below, I have taken fixed length of 100, taking character one by one till 
the point I reach the 100th character in the sentence (I am pre-padding those 
sequences where the length is less than 100) and after 100th character I start 
taking sliding window of 100.  
'''
seq_len = 100
dataX = []
dataY = []
for line in raw_text:
  if len(line) <= 100:
#     char_list = list(line)
    for i in range(1, len(line)):
      seq_in = line[:i]
      seq_out = line[i]
      dataX.append([char_to_int[char] for char in seq_in])
      dataY.append(char_to_int[seq_out])
  else:
#     num_lines = np.ceil(len(line)/seq_len)
    for i in range(1, len(line)):
      if i <= 100:
        seq_in = line[:i]
        seq_out = line[i]
      else:
        seq_in = line[(i-seq_len):i]
        seq_out = line[i]
      dataX.append([char_to_int[char] for char in seq_in])
      dataY.append(char_to_int[seq_out])
dataX = pad_sequences(dataX, maxlen=seq_len, padding='pre', value=0)  

In [9]:
print(len(dataX), len(dataX[1]))
print(len(dataY))

151352 100
151352


In [10]:
X = np.reshape(dataX, (len(dataX), seq_len, 1))
X = X/float(n_vocab)
y = np_utils.to_categorical(dataY)
print(X.shape)
print(y.shape)

(151352, 100, 1)
(151352, 38)


In [11]:
# define the LSTM model
model = Sequential()
model.add(InputLayer(input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.summary()

W0726 15:03:28.304238 140416076732288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 15:03:28.353207 140416076732288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 15:03:28.360413 140416076732288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0726 15:03:28.379365 140416076732288 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a futur

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_1 (Dropout)          (None, 100, 1)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 38)                9766      
Total params: 799,270
Trainable params: 799,270
Non-trainable params: 0
_________________________________________________________________


In [13]:
'''
Due to lack of time, I am only training for 7 epohcs as of now. I'll submit the 
assignment again on 27th July when the training finishes for 100 epochs.
'''
model.compile(loss='categorical_crossentropy', optimizer='adam')

save_dir = os.path.join(os.getcwd(), 'assgn2_saved_models')
model_name = 'lstm_model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='acc',
                             verbose=1,
                             save_best_only=True, period=5)

callbacks = [checkpoint]
model.fit(X, y, batch_size=128, epochs=7, callbacks=callbacks)

Epoch 1/7
151352/151352 [==============================] - 965s 6ms/step - loss: 2.7207
Epoch 2/7
151352/151352 [==============================] - 970s 6ms/step - loss: 2.5354
Epoch 3/7
151352/151352 [==============================] - 969s 6ms/step - loss: 2.3718
Epoch 4/7
151352/151352 [==============================] - 951s 6ms/step - loss: 2.2619
Epoch 5/7
151352/151352 [==============================] - 966s 6ms/step - loss: 2.1835
Epoch 6/7


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


151352/151352 [==============================] - 968s 6ms/step - loss: 2.1213
Epoch 7/7
151352/151352 [==============================] - 968s 6ms/step - loss: 2.0660


In [14]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/EIP3")

In [0]:
filename = "LSTM_epochs7.h5"
save_dir = os.path.join(os.getcwd(), 'assgn2_saved_models')
filepath = os.path.join(save_dir, filename)

model.save(filepath)

In [26]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
"                 i never went to him the mock turtle said with a sigh he taught laughing and grief th "


In [34]:
'''
Currently it generates only gibberish which is quite expected as 
I have only trained for only 7 epochs. I'll train for 100 epochs and
check what difference it makes.
'''
# generate characters
for i in range(500):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern =np.append(pattern, index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

 toe sf tuoet ontercerg tuoet on tu treee tf toeceed toe sf tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor tuoeer gor 
Done.


In [0]:
# Code for other approach that I was trying 
# padding sequences per batch didn't work because of too much ram consumption
# I'm using authors way of creating batches


# def generate_sequences(text, batch_size=4):
#     X_list = []
#     y_list = []
#     total_batches = len(text)/batch_size if len(text)%batch_size == 0 else (len(text)//batch_size+1)
#     for line_num in range(total_batches):
#         batch_lines = text[line_num*batch_size : (line_num+1)*batch_size]
#         x = [] 
#         y = []
#         for line in batch_lines:
#             char_list = list(line)
#             for i in range(1, (len(char_list)-1)):
#                 seq_in = char_list[:i]
#                 seq_out = char_list[i+1]
#                 x.append([char_to_int[char] for char in seq_in])
#                 y.append(char_to_int[seq_out])
# #                 print(x)
#         max_sequence_len = max(list(map(len, x)))
#         x = np.array(pad_sequences(x, maxlen=max_sequence_len, padding="pre", value=-10))
#         x = x[:, :, np.newaxis]
#         y = ku.to_categorical(y, num_classes=n_vocab)
#         X_list.append(x)
#         y_list.append(y)
#     return X_list, y_list   
  
  
# X, y = generate_sequences(raw_text)
# print(X[1].shape)
# print(y[1].shape)

# print(len(X))

# class My_Generator(ku.Sequence):
#     'Generates data for Keras'
#     def __init__(self, X, y, batch_size=1, shuffle=True):
#         'Initialization'
#         self.X = X
#         self.y = y
#         self.batch_size = batch_size
#         self.shuffle = shuffle
#         self.on_epoch_end()

#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return int(np.ceil(len(self.y)/self.batch_size))

#     def __getitem__(self, index):
#         return self.__data_generation(index)
        

#     def on_epoch_end(self):
#         'Shuffles indexes after each epoch'
#         self.indexes = np.arange(len(self.y))
#         if self.shuffle == True:
#             np.random.shuffle(self.indexes)

#     def __data_generation(self, index):
            
#         batch_x = self.X[index * self.batch_size:(index+1) * self.batch_size]
#         batch_y = self.y[index * self.batch_size:(index+1) * self.batch_size]
        
#         return batch_x, batch_y